In [1]:
from utils.data_cleaning import *
from utils.data_preprocess import *
from utils.evaluate import evaluate_model
from utils.cf_utils import *
from utils.models import *
from utils.big_data_helper import *
import polars as pl

from utils.maps_creater import *


import faiss
from tqdm import tqdm
from sklearn.pipeline import Pipeline


/opt/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data

### DataLoding

In [30]:
data = pl.scan_parquet("data/multi_event.parquet")

In [3]:
import polars as pl

estimate_parquet_ram_usage("data/multi_event.parquet")


📁 Файл: data/multi_event.parquet
📊 Сэмпл строк: 10,000
🔹 Размер sample: 0.19 MB
🔹 Всего строк в файле: 47,790,449

📐 Примерная оценка объёма данных в памяти: 0.90 GB


### Train/test split

Для начала разделим данные на обучающую и тестовую выборки. Деление выполняется по временному признаку, поскольку мы работаем с последовательностями действий пользователей.

Чтобы избежать data leakage — ситуации, когда признаки, рассчитанные на исторических данных, могут «заглядывать» в будущее — вводится временной зазор (gap) между обучающей и тестовой частями.

В оригинальной статье https://www.arxiv.org/pdf/2505.22238  используется соотношение 300 дней обучения, 30 минут gap, 1 день теста.

В моих экспериментах наилучшие результаты получились при конфигурации: 80 дней обучения, 30 минут gap, 20 дней теста.

Чтобы приблизиться к условиям статьи, я планирую снижать размер тестовой выборки, сохраняя возможность варьировать объём обучающей части как гиперпараметр модели. Такой подход позволяет гибко контролировать баланс между длиной обучающего периода и реалистичностью оценки качества.

In [31]:
train_lf, test_lf = train_test_split(data, 1, gap_size = 30)
train_lf.sink_parquet("data/work/train_df.parquet")
test_lf.sink_parquet("data/work/test_df.parquet")

In [5]:
estimate_parquet_ram_usage("data/work/train_df.parquet")
estimate_parquet_ram_usage("data/work/test_df.parquet")


📁 Файл: data/work/train_df.parquet
📊 Сэмпл строк: 10,000
🔹 Размер sample: 0.19 MB
🔹 Всего строк в файле: 47,550,037

📐 Примерная оценка объёма данных в памяти: 0.90 GB

📁 Файл: data/work/test_df.parquet
📊 Сэмпл строк: 10,000
🔹 Размер sample: 0.19 MB
🔹 Всего строк в файле: 238,398

📐 Примерная оценка объёма данных в памяти: 0.00 GB


### Очиска и кодирование данных 
Применим необходимую предобработку для очистки и подготовки данных. Подробные рассуждения о шагах и мотивации предобработки приведены в блокноте multi_event_preprocessing.
Все функции вынесены в отдельный модуль, что делает код более читаемым, модульным и удобным для повторного использования.

Кодируем uid и item_id в числовые индексы, чтобы модели и алгоритмы могли с ними работать.

Строковые ID нельзя напрямую подать в матрицу взаимодействий или Embedding-слой, поэтому мы переводим их в диапазон 0..N.
Это нужно для эффективности (память, скорость), совместимости с моделями (Matrix Factorization, Embeddings) и стабильного маппинга между train и test.

In [96]:
train_lf = pl.scan_parquet("data/work/train_df.parquet")
test_lf = pl.scan_parquet("data/work/test_df.parquet")

In [2]:
# создаём словари индексов
user_map, item_map = build_id_maps(train_lf)

# мапим train/test
train_encoded_lf = map_with_id_maps(train_lf, user_map, item_map)
test_encoded_lf  = map_with_id_maps(test_lf,  user_map, item_map)

train_encoded_lf.sink_parquet("data/work/train_encoded_lf.parquet")
test_encoded_lf.sink_parquet("data/work/test_encoded_lf.parquet")

NameError: name 'train_lf' is not defined

#### Сохраняем мар-ы для повторного кодирования

In [45]:
import json

with open("data/item_map.json", "w", encoding="utf-8") as f:
    json.dump(item_map, f, ensure_ascii=False, indent=2)

with open("data/user_map.json", "w", encoding="utf-8") as f:
    json.dump(item_map, f, ensure_ascii=False, indent=2)

In [46]:
apply_function_by_batch("data/work/train_encoded_lf.parquet", 
                         "data/work/train_df_listen.parquet", 
                         get_listen_data, 
                         "timestamp", 
                         batch_size = 10_000_000)

apply_function_by_batch("data/work/test_encoded_lf.parquet", 
                         "data/work/test_df_listen.parquet", 
                         get_listen_data , 
                         "timestamp", 
                         batch_size = 10_000_000)

apply_function_by_batch("data/work/train_encoded_lf.parquet", 
                         "data/work/train_df_likes.parquet", 
                         get_not_listen_data, 
                         "timestamp", 
                         batch_size = 10_000_000)

apply_function_by_batch("data/work/test_encoded_lf.parquet", 
                         "data/work/test_df_likes.parquet", 
                         get_not_listen_data , 
                         "timestamp", 
                         batch_size = 10_000_000)

In [47]:
# Из трейна remove_duplicates_by_timestamps
apply_function_by_batch("data/work/train_df_listen.parquet", 
                         "data/work/train_df_listen_1.parquet", 
                         remove_duplicates_by_timestamps , 
                         "timestamp", 
                         batch_size = 10_000_000)

# Из трейна filter_rare_items
apply_function_by_batch("data/work/train_df_listen_1.parquet", 
                         "data/work/train_df_listen_2.parquet", 
                         filter_rare_items , 
                         "item_id", 
                         batch_size = 10_000_000)

# Из трейна filter_rare_users
apply_function_by_batch("data/work/train_df_listen_2.parquet", 
                         "data/work/train_df_listen_3.parquet", 
                         filter_rare_users , 
                         "uid", 
                         batch_size = 10_000_000)

# Из трейна cut_track_len
apply_function_by_batch("data/work/train_df_listen_3.parquet", 
                         "data/work/train_df_listen_4.parquet", 
                         cut_track_len , 
                         "timestamp", 
                         batch_size = 10_000_000)

In [48]:
# Из трейна convert_reaction
apply_function_by_batch("data/work/train_df_likes.parquet", 
                         "data/work/train_df_likes_1.parquet", 
                         convert_reaction , 
                         "uid", 
                         batch_size = 10_000_000)

# Из трейна rename_events
apply_function_by_batch("data/work/train_df_likes_1.parquet", 
                         "data/work/train_df_likes_2.parquet", 
                         rename_events , 
                         "timestamp", 
                         batch_size = 10_000_000)

In [49]:
# Из теста convert_reaction
apply_function_by_batch("data/work/test_df_likes.parquet", 
                         "data/work/test_df_likes_1.parquet", 
                         convert_reaction , 
                         "uid", 
                         batch_size = 10_000_000)

# Из теста rename_events
apply_function_by_batch("data/work/test_df_likes_1.parquet", 
                         "data/work/test_df_likes_2.parquet", 
                         rename_events , 
                         "timestamp", 
                         batch_size = 10_000_000)

In [50]:
concat_files("data/work/test_df_listen.parquet", "data/work/test_df_likes_2.parquet", "data/work/test_df_preprocessed.parquet")

concat_files("data/work/train_df_listen_4.parquet", "data/work/train_df_likes_2.parquet", "data/work/train_df_preprocessed.parquet")


In [2]:
estimate_parquet_ram_usage("data/work/train_df_preprocessed.parquet")
estimate_parquet_ram_usage("data/work/test_df_preprocessed.parquet")


📁 Файл: data/work/train_df_preprocessed.parquet
📊 Сэмпл строк: 10,000
🔹 Размер sample: 0.27 MB
🔹 Всего строк в файле: 31,907,792

📐 Примерная оценка объёма данных в памяти: 0.83 GB

📁 Файл: data/work/test_df_preprocessed.parquet
📊 Сэмпл строк: 10,000
🔹 Размер sample: 0.27 MB
🔹 Всего строк в файле: 237,786

📐 Примерная оценка объёма данных в памяти: 0.01 GB


#### Map для "уже послушанных песен пользователя"

Так как цель предсказывать что-то новое, что нужно фильтровать песни которые пользователь уже видел из рекомендаций, а так же не учитывать их при тестировании. При тестировании список прослушанных песен пользователя должен фильтровался от уже прослушанных, так как система не предсказывает такие песни и нет смысла учитывать их при тестировании. Пользователи которые не слушают новые песни не включаются в оценки (модель не может угадывать то, чего не было). Для учета таких случаев будет введена отдельная метрика, показывающая вовлеченность пользователей в рекомендательную систему.

In [2]:
# Береме данные до препроцессинка (но закодированные)
history =  pl.scan_parquet("data/work/train_encoded_lf.parquet")

test_df = pl.scan_parquet("data/work/test_df_preprocessed.parquet")
test_users_items_df = remove_listened_data(test_df)

test_users_items_df.sink_parquet("data/work/test_df_preprocessed_for_eval.parquet")


In [3]:
train_df = pl.scan_parquet("data/work/train_df_preprocessed.parquet")
test_df  = pl.scan_parquet("data/work/test_df_preprocessed_for_eval.parquet") 
history =  pl.scan_parquet("data/work/train_encoded_lf.parquet")
users_history = build_users_history(history, last_days=30)

# Models

## Baseline

#### Глобальные тренды

Самый простой базовый подход — взять топ-10 самых популярных треков. При этом возможны два варианта отбора:

* по количеству лайков,
* по количеству прослушиваний.

Реализованы обе версии, однако в качестве базовой модели (baseline) выбран именно топ-10 самых залайканных треков среди всех пользователей. Лайки считаются более надёжным сигналом предпочтений, чем прослушивания. (Для чистоты эксперимента был протестирован и вариант с прослушиваниями — его точность оказалась ниже.)



Поскольку тренды и популярные песни сменяются очень быстро, временное окно, в котором мы определяем тренд, также должно быть коротким. (*PS нужно оптимизировать выбор автоматически*)

In [5]:
model = MostPop_by_likes()
model.fit(train_df)
evaluate_model(model, users_history, test_df)

100%|██████████████████████████████████████████████████████████████████████████████| 4827/4827 [00:00<00:00, 53293.56it/s]

Mean Recall@10: 0.014374850154158968
Mean NDCG@10: 0.02529114756567746


In [6]:
model = MostPop_by_listen()
model.fit(train_df)

evaluate_model(model, users_history, test_df)

100%|██████████████████████████████████████████████████████████████████████████████| 4827/4827 [00:00<00:00, 52055.66it/s]

Mean Recall@10: 0.01248428881344854
Mean NDCG@10: 0.02363284633339717


#### Новинки последних дней

Похоже на модель популярных песен, но берет только новые песни. В топ прослушаных могут попадать к примеру песня Шуфутинского 3го сентября 

In [7]:
model = NewItemsLastNDays()
model.fit(train_df)

evaluate_model(model, users_history, test_df)

100%|██████████████████████████████████████████████████████████████████████████████| 4827/4827 [00:00<00:00, 51106.28it/s]

Mean Recall@10: 0.006657907090237797
Mean NDCG@10: 0.015225396394740779


#### Последнее прослушанное

Скорее всего пользователь в ближайщем времени послушает то, что слушал некоторое время назад. Такое предсказние дает большой процент точности, но не совсем то что нужно для рекомендательной системы, у которой цель посоветовать что-то новое и не из последний песен плейлиста. Такие песни можно будет исключать в будущем 

In [104]:
test_df  = pl.scan_parquet("data/work/test_df_preprocessed_for_eval.parquet") 
train_df = pl.scan_parquet("data/work/train_df_preprocessed.parquet")

model = LastListenRecommender()


hour = 24
decay = 0.9
tau = 0.0 if hour == 0 else decay ** (1 / 24 / 60 / 60 / (hour / 24))

df_tau = add_exponential_decay(train_df, tau)

model.fit(df_tau)

evaluate_model(model, users_history, test_df, 50) # 50 потому что в планах фильтровать последние 50 прослушанных 

100%|████████████████████████████████████████████████████████████████████████████████| 4827/4827 [00:20<00:00, 233.11it/s]

Mean Recall@50: 0.23095763187369045
Mean NDCG@50: 0.20866173981174777


## Collaborative filtering

### ALS

AlternatingLeastSquares (ALS) — это модель матричной факторизации для неявных откликов (implicit feedback).
Она раскладывает матрицу взаимодействий пользователей и объектов на две матрицы эмбеддингов — пользователей и айтемов — так, чтобы их скалярное произведение предсказывало интерес.
Обучается поочерёдным решением задач наименьших квадратов: сначала фиксирует айтемы и обновляет пользователей, потом наоборот.


В одной итерации ALS поочерёдно пересчитывает вектора пользователей и айтемов.
Сначала, при фиксированных айтемах, решаются задачи наименьших квадратов для всех пользователей;
потом, при фиксированных пользователях — для всех айтемов.
Это повторяется несколько итераций, пока модель не сойдётся.

Для обучения ALS требуется подготовить данные в формате матрицы взаимодействий, где каждая ячейка отражает степень интереса пользователя к объекту. Поскольку в нашем случае используются implicit-сигналы, нам необходимо преобразовать исходные события (прослушивание, лайк, дизлайк) в числовые веса.

На этапе подготовки данных мы подсчитываем, сколько раз пользователь прослушал каждый трек, а также приводим реакции к бинарному виду:
в результирующем датафрейме поле like получает значение 1,
аналогично dislike — значение 1.

В отдельном модуле реализована функция объединения данных, которая формирует итоговый датафрейм с колонками:

* признак лайка (like = 1)
* признак дизлайка (dislike = 1)
* количество прослушиваний каждой пары пользователь–трек

In [8]:

train_df_conf = merge_data_by_count(train_df)
train_df_conf = calculate_conf(train_df_conf)

(
    train_df_conf
    .collect()  # выполняем план, получаем обычный DataFrame
    .write_parquet("data/work/train_df_conf.parquet")
)

In [9]:
estimate_parquet_ram_usage("data/work/train_df_conf.parquet")


📁 Файл: data/work/train_df_conf.parquet
📊 Сэмпл строк: 10,000
🔹 Размер sample: 0.36 MB
🔹 Всего строк в файле: 7,298,676

📐 Примерная оценка объёма данных в памяти: 0.26 GB


Для ALS необходима разреженная матрица вида:

(user_id, item_id) → weight


где weight — это итоговая сила взаимодействия (прослушивания + реакции).
В модуле реализована функция, которая на основе сформированного датафрейма строит такую матрицу через coo_matrix и возвращает её в подходящем для модели формате.

In [204]:
train_df_conf = pl.scan_parquet("data/work/train_df_conf.parquet")

train_df_conf = train_df_conf.with_columns(
    (pl.col("conf") != 0).cast(pl.Int8).alias("conf")
)

model = ALS()
model.fit(train_df_conf)



/opt/venv/lib/python3.11/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 16 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()
100%|███████████████████████████████████████████████████████████████████████| 15/15 [00:27<00:00,  1.84s/it, loss=0.00264]


In [205]:
evaluate_model(model, users_history, test_df)

100%|█████████████████████████████████████████████████████████████████████████████████| 4827/4827 [01:53<00:00, 42.65it/s]


Mean Recall@10: 0.017005667891855
Mean NDCG@10: 0.04118527317112505


Метод ALS работает только с пользователями и объектами, которые присутствовали в обучающей выборке. Если в тесте встречается пользователь, которого модель ранее не видела, для него невозможно построить профиль в факторизационном пространстве.

В такой ситуации ALS не способен выдать рекомендации — в классе ALS такая логика приводит к возврату пустого списка.

Поэтому при использовании ALS важно учитывать, что:

* рекомендации гарантированно работают только для пользователей из train,
* все новые или ранее невстречавшиеся пользователи будут давать «пустой» результат.

Это естественное ограничение моделей матричной факторизации, основанных на implicit-feedback данных.

### BM25Rec

In [75]:
model = BM25Rec()
model.fit(train_df_conf)
evaluate_model(model, users_history, test_df)


### ItemKNN

In [209]:
model = ItemKNN()
model.fit(train_df)
evaluate_model(model, users_history, test_df, k=10)

100%|█████████████████████████████████████████████████████████████████████████████████| 4827/4827 [01:46<00:00, 45.44it/s]

Mean Recall@10: 0.0431855077752173
Mean NDCG@10: 0.08053783579665744


 В памяти все хранить очень тяжело, но по математическим расчетам это тоже самое, поэтому используем
 Конечно тоже не быстро но лучше чем было

### BPR

In [210]:
train_df_conf = pl.scan_parquet("data/work/train_df_conf.parquet")

train_df_conf = train_df_conf.with_columns(
    (pl.col("conf") != 0).cast(pl.Int8).alias("conf")
)

model = BPR()
model.fit(train_df_conf)

evaluate_model(model, users_history, test_df)

100%|█████████████████████████████████████████████████████████████████████████████████| 4827/4827 [01:57<00:00, 41.08it/s]

Mean Recall@10: 0.014311525458997233
Mean NDCG@10: 0.030514844986110252


## Content_based 

Content-Based Filtering — это метод рекомендаций, который подбирает объекты, похожие на те, что пользователь уже оценил, используя их содержательные признаки (жанры, embedding-и, особенности контента). Его сильная сторона в том, что он не зависит от других пользователей, хорошо работает для новых объектов (нет cold-start для items) и даёт легко объяснимые рекомендации. Однако у подхода есть ограничения: качество сильно зависит от самих признаков, модель не умеет учитывать тренды или контекст поведения, а рекомендации могут быть слишком однообразными и «сужать» вкусы пользователя.


### масенькая моделька по артистам и альбомам

In [2]:
import json 
train_df = pl.scan_parquet("data/work/train_df_preprocessed.parquet")


df_merged = merge_data_by_count(train_df)


with open("data/item_map.json", "r", encoding="utf-8") as f:
    item_map = json.load(f)

item_map = {int(k): v for k, v in item_map.items()}

items_meta = (
    pl.scan_parquet("data/items_meta.parquet")
    .with_columns(
        pl.col("item_id").replace(item_map)
    )
    .unique(subset=["item_id"])
    .drop_nulls()
)

lf_user_track = df_merged.join(
    items_meta.select(["item_id", "artist_id", "album_id"]),
    on="item_id",
    how="left",
)


lf_user_track.collect()

uid,item_id,listen_count,played_ratio_max,dislike_flag,like_flag,artist_id,album_id
i64,i64,u32,u16,i32,i32,f64,f64
7733,390393,1,15,0,0,373262.0,734649.0
4222,8779,1,100,0,0,95334.0,929085.0
5283,637630,9,100,0,1,840572.0,2.950615e6
9805,575892,3,100,0,0,241913.0,1.211641e6
2482,712581,1,100,0,0,766195.0,685589.0
…,…,…,…,…,…,…,…
8878,834301,0,0,0,1,83803.0,163486.0
3973,563761,0,0,0,1,10557.0,2.583746e6
3172,792296,0,0,0,1,66643.0,2.414815e6


In [3]:
import numpy as np
import polars as pl
from scipy import sparse

# --------------------------------------------------------
# 1. Сбор данных из Polars и подготовка рёбер
# --------------------------------------------------------

# lf_user_track — твой LazyFrame с колонками:
# uid, item_id, artist_id, album_id, listen_count
df = lf_user_track.collect()

# --- user–track с весами (1 + log1p(listen_count)) ---
df_ut = (
    df
    .select(
        "uid",
        "item_id",
        (1 + pl.col("listen_count").log1p()).alias("w")
    )
    .group_by(["uid", "item_id"])
    .agg(pl.col("w").sum().alias("weight"))
)

# --- track–artist, вес = 1 ---
df_ta = (
    df
    .select("item_id", "artist_id")
    .unique()
    .with_columns(pl.lit(1.0).alias("weight"))
)

# --- track–album, вес = 1 ---
df_tal = (
    df
    .select("item_id", "album_id")
    .unique()
    .with_columns(pl.lit(1.0).alias("weight"))
)

# --------------------------------------------------------
# 2. Кодирование узлов в int и построение CSR-матрицы
# --------------------------------------------------------

# Уникальные ID по типам узлов
user_ids = df["uid"].unique().to_list()
track_ids = df["item_id"].unique().to_list()
artist_ids = df["artist_id"].unique().to_list()
album_ids = df["album_id"].unique().to_list()

n_users = len(user_ids)
n_tracks = len(track_ids)
n_artists = len(artist_ids)
n_albums = len(album_ids)

# Смещения для каждого типа узла
offset_user = 0
offset_track = offset_user + n_users
offset_artist = offset_track + n_tracks
offset_album = offset_artist + n_artists

n_nodes = n_users + n_tracks + n_artists + n_albums

# Маппинги ID → индекс узла
user2idx = {u: offset_user + i for i, u in enumerate(user_ids)}
track2idx = {t: offset_track + i for i, t in enumerate(track_ids)}
artist2idx = {a: offset_artist + i for i, a in enumerate(artist_ids)}
album2idx = {al: offset_album + i for i, al in enumerate(album_ids)}

# --- user–track рёбра ---
u_arr = df_ut["uid"].to_list()
t_arr_ut = df_ut["item_id"].to_list()
w_ut = df_ut["weight"].to_numpy()

rows_ut = np.fromiter((user2idx[u] for u in u_arr), dtype=np.int64, count=len(u_arr))
cols_ut = np.fromiter((track2idx[t] for t in t_arr_ut), dtype=np.int64, count=len(t_arr_ut))

# --- track–artist рёбра ---
t_arr_ta = df_ta["item_id"].to_list()
a_arr = df_ta["artist_id"].to_list()
w_ta = df_ta["weight"].to_numpy()

rows_ta = np.fromiter((track2idx[t] for t in t_arr_ta), dtype=np.int64, count=len(t_arr_ta))
cols_ta = np.fromiter((artist2idx[a] for a in a_arr), dtype=np.int64, count=len(a_arr))

# --- track–album рёбра ---
t_arr_tal = df_tal["item_id"].to_list()
al_arr = df_tal["album_id"].to_list()
w_tal = df_tal["weight"].to_numpy()

rows_tal = np.fromiter((track2idx[t] for t in t_arr_tal), dtype=np.int64, count=len(t_arr_tal))
cols_tal = np.fromiter((album2idx[al] for al in al_arr), dtype=np.int64, count=len(al_arr))

# Собираем все ориентированные рёбра
rows_dir = np.concatenate([rows_ut, rows_ta, rows_tal])
cols_dir = np.concatenate([cols_ut, cols_ta, cols_tal])
data_dir = np.concatenate([w_ut, w_ta, w_tal]).astype(np.float32)

# Делаем граф неориентированным: дублируем рёбра в обе стороны
rows_full = np.concatenate([rows_dir, cols_dir])
cols_full = np.concatenate([cols_dir, rows_dir])
data_full = np.concatenate([data_dir, data_dir])

# Разреженная матрица смежности A (CSR)
A = sparse.csr_matrix(
    (data_full, (rows_full, cols_full)),
    shape=(n_nodes, n_nodes),
    dtype=np.float32,
)

# --------------------------------------------------------
# 3. Матрица переходов P и вспомогательные структуры
# --------------------------------------------------------

# Нормируем строки A → стохастическая матрица переходов P
row_sums = np.array(A.sum(axis=1)).flatten().astype(np.float64)
row_sums[row_sums == 0.0] = 1.0  # чтобы избежать деления на 0

D_inv = sparse.diags(1.0 / row_sums)
P = D_inv @ A  # P: каждая строка — распределение перехода

# Для PPR удобнее использовать P^T
P_T = P.T.tocsr()

# Для рекомендаций пригодятся:
# - исходный df (чтобы исключать уже слушанные треки)
df_user_track = df  # просто переименуем для совместимости с твоим кодом

# --------------------------------------------------------
# 4. Функция Personalized PageRank
# --------------------------------------------------------

def personalized_pagerank(start_idx, alpha=0.85, tol=1e-6, max_iter=100):
    """
    PageRank с персонализацией на одном узле (start_idx).
    Возвращает вектор размерности n_nodes.
    """
    n = P_T.shape[0]

    # personalization-вектор v
    v = np.zeros(n, dtype=np.float32)
    v[start_idx] = 1.0

    # инициализация x
    x = np.full(n, 1.0 / n, dtype=np.float32)

    for _ in range(max_iter):
        x_new = alpha * (P_T @ x) + (1 - alpha) * v
        # L1-норма разности
        if np.linalg.norm((x_new - x).astype(np.float64), 1) < tol:
            x = x_new
            break
        x = x_new

    return x

# --------------------------------------------------------
# 5. Рекомендации для пользователя
# --------------------------------------------------------

def recommend_for_user(uid, top_k=10, exclude_listened=True, alpha=0.85):
    """
    Аналог твоей функции recommend_for_user, но на scipy.sparse.

    uid: ID пользователя в исходных данных (как в lf_user_track)
    top_k: сколько треков вернуть
    exclude_listened: исключать ли уже слушанные пользователем треки
    alpha: коэффициент затухания в PageRank
    """
    if uid not in user2idx:
        return []

    start_node_idx = user2idx[uid]

    # Считаем personalized PageRank
    scores = personalized_pagerank(start_node_idx, alpha=alpha)

    # Берём только треки (их индексы идут подряд от offset_track)
    track_scores = scores[offset_track : offset_track + n_tracks]

    # # Исключаем уже слушанные треки, если нужно
    # if exclude_listened:
    #     listened_track_ids = (
    #         df_user_track
    #         .filter(pl.col("uid") == uid)
    #         .select("item_id")
    #         .to_series()
    #         .to_list()
    #     )
    #     listened_set = set(listened_track_ids)

    #     # маска по track_ids (track_ids[i] — item_id трека с локальным индексом i)
    #     mask = np.array([tid not in listened_set for tid in track_ids], dtype=np.float32)
    #     track_scores = track_scores * mask

    # Top-k индексы
    if top_k < len(track_scores):
        # argpartition — быстрее, чем полная сортировка
        top_idx = np.argpartition(-track_scores, top_k)[:top_k]
        # досортируем по убыванию
        top_idx = top_idx[np.argsort(-track_scores[top_idx])]
    else:
        top_idx = np.argsort(-track_scores)

    # Возвращаем исходные item_id
    return [track_ids[i] for i in top_idx[:top_k]]

# --------------------------------------------------------
# 6. Пример использования
# --------------------------------------------------------

# пример
print(recommend_for_user(uid=200, top_k=200))


[749032, 97545, 665904, 131632, 825587, 867039, 662894, 797407, 671638, 799324, 255462, 872952, 241651, 272518, 585400, 266016, 265523, 514880, 711816, 573519, 627154, 736229, 205826, 734743, 613567, 445383, 90184, 231245, 51109, 379633, 782732, 691908, 397074, 225399, 810966, 143006, 566463, 351015, 338644, 358522, 604744, 741275, 107302, 511362, 397161, 752174, 526016, 472249, 615306, 815190, 345733, 766681, 369523, 252905, 705928, 920099, 833248, 348693, 756099, 180157, 574859, 245958, 457057, 822806, 547731, 27816, 663190, 306358, 883592, 401591, 21370, 43405, 851437, 760758, 109145, 533911, 616374, 249481, 849392, 134015, 777906, 603906, 298695, 176840, 71117, 132676, 683185, 552809, 426118, 58178, 128529, 284435, 414422, 516460, 34471, 659058, 698779, 759, 340478, 854564, 235821, 831612, 636972, 483667, 392926, 575087, 240937, 153064, 5461, 436776, 904309, 209117, 644945, 295349, 773060, 831163, 718963, 117168, 490350, 72209, 392307, 298755, 288259, 10064, 500386, 679924, 627037,

In [38]:
pred = recommend_for_user(uid=729, top_k=200)

In [37]:
real = pl.scan_parquet("data/work/test_df_preprocessed_for_eval.parquet").filter(pl.col("uid") == 729).select("items").collect().to_series().to_list()[0]  #28556

In [47]:
pl.scan_parquet("data/work/test_df_preprocessed.parquet").filter(pl.col("uid") == 729).filter(pl.col("item_id") == 855098).collect()  #28556

timestamp,is_organic,played_ratio_pct,track_length_seconds,event_type,uid,item_id
u32,u8,u16,u32,enum,i64,i64
25956365,0,100,100,"""listen""",729,855098


In [42]:
not_like = lf_user_track.filter(pl.col("played_ratio_max") < 50).filter(pl.col("uid") == 123).select(pl.col("item_id").unique()).collect().to_series().to_list()



In [54]:
set(pred[:50])&set(real)

{119158}

In [32]:
pl.scan_parquet("data/work/test_df_preprocessed.parquet").filter(pl.col("is_organic") == 0).collect()

timestamp,is_organic,played_ratio_pct,track_length_seconds,event_type,uid,item_id
u32,u8,u16,u32,enum,i64,i64
25951550,0,74,185,"""listen""",729,61
25958055,0,3,205,"""listen""",2460,127
25970010,0,86,205,"""listen""",3463,127
25939055,0,100,205,"""listen""",3463,127
25939710,0,87,205,"""listen""",1136,127
…,…,…,…,…,…,…
25937645,0,null,null,"""dislike""",9447,null
25998165,0,null,null,"""like""",8701,null
25997295,0,null,null,"""dislike""",3600,null


In [51]:
# Проверка, совпадают ли item_ids

items1 = set(model1.reverse_item_map.keys())
items2 = set(model2.user_map.keys())
items3 = set(model3.user_map.keys())

print("Совпадают ли item-ы model1 и model2:", items1 == items2)
print("Совпадают ли item-ы model1 и model3:", items1 == items3)
print("Пересечение model1 и model2:", len(items1 & items2))
print("Пересечение model1 и model3:", len(items1 & items3))

Совпадают ли item-ы model1 и model2: False
Совпадают ли item-ы model1 и model3: False
Пересечение model1 и model2: 122052
Пересечение model1 и model3: 122052


In [3]:
# model2.user_map[]

In [12]:
import numpy as np
import networkx as nx
from scipy.sparse import csr_matrix

########################################################
# 1. Допустим, есть такие матрицы
########################################################

# Пример: 3 пользователя, 5 треков
# R_ui[u, i] = 1, если пользователь u слушал трек i
R_ui = model1.user_item

num_users, num_items = R_ui.shape

# Матрица item-artist: I x A
I_A = model2.user_item

num_items2, num_artists = I_A.shape
assert num_items2 == num_items

# Матрица item-album: I x L
I_L = model3.user_item
num_items3, num_albums = I_L.shape
assert num_items3 == num_items



In [13]:
########################################################
# 2. Строим граф
########################################################

G = nx.Graph()

# Добавляем узлы (это не обязательно, но понятно)
for u in tqdm(range(num_users)):
    G.add_node(f"u{u}", type="user")

for i in tqdm(range(num_items)):
    G.add_node(f"i{i}", type="item")

for a in tqdm(range(num_artists)):
    G.add_node(f"a{a}", type="artist")

for l in tqdm(range(num_albums)):
    G.add_node(f"al{l}", type="album")

# Рёбра user–item из R_ui
u_idx, i_idx = R_ui.nonzero()
for u, i in tqdm(zip(u_idx, i_idx)):
    G.add_edge(f"u{u}", f"i{i}")

# Рёбра item–artist из I_A
i_idx, a_idx = I_A.nonzero()
for i, a in tqdm(zip(i_idx, a_idx)):
    G.add_edge(f"i{i}", f"a{a}")

# Рёбра item–album из I_L
i_idx, l_idx = I_L.nonzero()
for i, l in tqdm(zip(i_idx, l_idx)):
    G.add_edge(f"i{i}", f"al{l}")


100%|█████████████████████████████████████████████████████████████████████████| 141706/141706 [00:00<00:00, 562975.00it/s]
4159021it [00:22, 184804.32it/s]
337067it [00:03, 96194.25it/s] 
337067it [00:01, 202626.20it/s]


In [26]:
########################################################
# 3. Рекомендации на основе Personalized PageRank
########################################################

# Рекомендации для пользователя на основе Personalized PageRank
def recommend_for_user(user_id, top_k=10):
    user_node = f"u{user_id}"

    # Задаём персонализацию, только пользовательский узел равен 1
    personalization = {n: 0.0 for n in G.nodes()}
    personalization[user_node] = 1.0

    # Запускаем Personalized PageRank
    ppr = nx.pagerank(G, alpha=0.85, personalization=personalization)

    # Выбираем только айтемы, которые пользователь еще не слушал
    item_scores = {}
    for node, score in ppr.items():
        if node.startswith("i"):
            i = int(node[1:])
            if R_ui[user_id, i] == 0:
                item_scores[i] = score

    ranked_items = sorted(item_scores.items(), key=lambda x: x[1], reverse=True)
    top_items = [i for i, _ in ranked_items[:top_k]]
    top_scores = ranked_items[:top_k]
    return top_items, top_scores

# Пример вызова
items, with_scores = recommend_for_user(0, top_k=10)
print("Рекомендованные треки для user 0:", items)
print("Скоры:", with_scores)


Рекомендованные треки для user 0: [103716, 201546, 93495, 179541, 138148, 83369, 108531, 268848, 76678, 124384]
Скоры: [(103716, 0.0001627225750999019), (201546, 0.00014191306367744572), (93495, 0.00011686221659254939), (179541, 0.00011612150216219119), (138148, 0.00011609270334734047), (83369, 0.00011395351214372441), (108531, 0.00011012176499994559), (268848, 0.00010968250796352267), (76678, 0.00010506156026028385), (124384, 0.00010485918121808665)]


In [36]:
model1.reverse_item_map[201546]


# [103716, 201546, 93495, 179541, 138148, 83369, 108531, 268848, 76678, 124384]

445383

In [37]:
pl.scan_parquet("data/work/train_encoded_lf.parquet").filter(pl.col("uid") == 1849).filter(pl.col("event_type") == "listen").filter(pl.col("item_id") == 445383).collect()  #28556

timestamp,is_organic,played_ratio_pct,track_length_seconds,event_type,uid,item_id
u32,u8,u16,u32,enum,i64,i64
20368345,1,1,220,"""listen""",1849,445383


In [ ]:
[[53808,
  95865,
  118912,
  97059,
  153986,
  198175,
  264863,
  266016,
  241651,
  450734,
  400230,
  420720,
  401977,
  797407,
  811597,
  571012,
  748916,
  732593,
  665904,
  908037,
  481524,
  498278,
  503065]]

#### embeddongs 

In [7]:
import json

with open("data/item_map.json", "r", encoding="utf-8") as f:
    item_map = json.load(f)

item_map = {int(k): v for k, v in item_map.items()}

Методы точного поиска ближайших соседей (kNN) требуют вычислять расстояние между вектором-запросом и каждым объектом в базе, поэтому их сложность линейная 
𝑂
(
𝑁
)
O(N), и на больших наборах данных с десятками или сотнями тысяч элементов они становятся крайне медленными: нужно выполнить миллионы скалярных произведений при каждом запросе. По этой причине в рекомендациях и поиске похожих объектов чаще используют ANN (Approximate Nearest Neighbors) — алгоритмы, которые строят специальный индекс (HNSW, Annoy, Faiss и др.) и ищут ближайших соседей не полностью перебирая весь набор, а умно обходя лишь небольшую часть точек. В результате скорость поиска ускоряется в десятки–сотни раз при минимальной потере качества, что делает ANN практически стандартом для больших эмбеддингов.

In [8]:
def create_index(filename, item_id_map):
    lf = pl.scan_parquet(filename)
    CHUNK_SIZE = 50_000
    
    # Возьмём размерность D из первого батча
    first_batch = (
        lf
        .slice(0, CHUNK_SIZE)
        .select(["item_id", "normalized_embed"])
        .collect()
        .to_pandas()
    )

    first_batch["item_id"] = first_batch["item_id"].map(item_id_map)
    
    first_batch = first_batch.dropna()    

    
    emb0 = np.vstack(first_batch["normalized_embed"].to_list()).astype("float32")
    N0, D = emb0.shape
    
    index = faiss.IndexFlatIP(D)   # или что-то более сложное, см. ниже
    index.add(emb0)
    
    all_item_ids = []
    all_item_ids.extend(first_batch["item_id"].tolist())
    
    # Теперь итерируемся по файлу батчами
    offset = CHUNK_SIZE
    while True:
        batch = (
            lf
            .slice(offset, CHUNK_SIZE)
            .select(["item_id", "normalized_embed"])
            .collect()
            .to_pandas()
        )
    
        if batch.empty:
            break
    
        batch["item_id"] = batch["item_id"].map(item_id_map)
        batch = batch.dropna()
    
        if batch.empty:
            offset += CHUNK_SIZE
            continue
    
        emb_batch = np.vstack(batch["normalized_embed"].to_list()).astype("float32")
        index.add(emb_batch)
        all_item_ids.extend(batch["item_id"].tolist())
    
        offset += CHUNK_SIZE
        print(f"added up to offset={offset}, total indexed={index.ntotal}")

    return index, all_item_ids

In [9]:
index, all_item_ids = create_index("data/filtered_embeddings.parquet", item_map)

added up to offset=100000, total indexed=99757
added up to offset=150000, total indexed=149640
added up to offset=200000, total indexed=199532
added up to offset=250000, total indexed=249413
added up to offset=300000, total indexed=299301
added up to offset=350000, total indexed=349176
added up to offset=400000, total indexed=399045
added up to offset=450000, total indexed=448925
added up to offset=500000, total indexed=498808
added up to offset=550000, total indexed=548684
added up to offset=600000, total indexed=598557
added up to offset=650000, total indexed=648439
added up to offset=700000, total indexed=698323
added up to offset=750000, total indexed=748229
added up to offset=800000, total indexed=798118
added up to offset=850000, total indexed=845905


In [10]:
def build_users_history_normal(lf: pl.LazyFrame) -> dict:
    """
    Возвращает dict: uid -> set(item_id) только за последние 2 месяца.
    """

    history_df = (
        lf
        .select(["uid", "item_id"])
        .unique()
        .group_by("uid")
        .agg(pl.col("item_id").alias("items"))
        .collect()
    )

    return {
        row["uid"]: set(row["items"])
        for row in history_df.iter_rows(named=True)
    }

In [113]:
class CBF_by_embeding:
    def __init__(self):
        pass

    def fit(self, data, index, item_ids):
        self.data = data

        # self.data = self.data.filter(pl.col("listen_count")>20)

        self.index, self.item_ids = index, item_ids
        self.id2pos = {iid: i for i, iid in enumerate(self.item_ids)} # Индексы реальные и в матрице - разные. Нужно для сопоставления

        
        self.total = []
        self.N = 200
        
        self.history = build_users_history_normal(self.data)
        
    # Профиль пользователя = средний вектор всех айтемов, которые он слушал.
    def build_user_profile_embed(self, uid):

        if uid in self.history:
            
            listened_items = self.history[uid]
        else:
            self.total.append(1)
            return None
    
        indices = [self.id2pos[iid] for iid in listened_items if iid in self.id2pos]
        
        
        if len(indices) <1:
            
            return None
        
        
        vectors = np.vstack([self.index.reconstruct(int(i)) for i in indices]).astype("float32")

        user_vec = np.median(vectors, axis=0)
        user_vec = np.asarray(user_vec)        
                
        return np.array([user_vec])

    def similar_tracks(self, vec): #

        sims, ids = self.index.search(vec, self.N+1)
    
        sims = sims[0]
        ids = ids[0]
        
        result = []
        for track_pos, score in zip(ids, sims):
            result.append((self.item_ids[track_pos], float(score)))

        return [pair[0] for pair in result], [pair[1] for pair in result] 

    def recommend(self, uid):
        vec = self.build_user_profile_embed(uid)
        if vec is None:
            return [], []    
        return self.similar_tracks(vec)

In [10]:
import polars as pl

def build_train_last_n_by_listen_count(
    train_df: pl.LazyFrame | pl.DataFrame,
    min_listen_count: int = 5,
    top_k: int = 2,
):

    listens = train_df.filter(pl.col("event_type") == "listen").filter(pl.col("played_ratio_pct") > 90)

    agg = (
        listens
        .group_by(["uid", "item_id"])
        .agg([
            pl.col("timestamp").count().alias("listen_count"),
            pl.col("timestamp").max().alias("last_ts"),
        ])
    )

    agg = agg.filter(pl.col("listen_count") > min_listen_count)

    agg = agg.sort(["uid", "listen_count"], descending=[False, True])
    
    agg_ranked = agg.with_columns(
        pl.int_range(0, pl.count()).over("uid").alias("rank")
    )
    
    top_items = agg_ranked.filter(pl.col("rank") < top_k).drop("rank")
    
    return top_items


In [193]:
# tmp_df = pl.scan_parquet("data/train_encoded_lf.parquet")
train_df_conf_last_10days = build_train_last_n_by_listen_count(train_df, 20, 1)

# (
#     train_df_conf_last_10days
#     .collect()  # выполняем план, получаем обычный DataFrame
#     .write_parquet("data/work/train_df_conf_last_10days.parquet")
# )

/tmp/ipykernel_17545/1901212513.py:25: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.int_range(0, pl.count()).over("uid").alias("rank")


In [187]:
model = CBF_by_embeding()
model.fit(train_df_conf_last_10days, index, all_item_ids)

In [196]:
model.N = 200
evaluate_model(model, users_history, test_df)

100%|████████████████████████████████████████████████████████████████████████████████| 2884/2884 [00:24<00:00, 115.83it/s]

Mean Recall@10: 0.000951505801646164
Mean NDCG@10: 0.004019473352175542


In [ ]:
Низкая точность 

In [179]:

# pl.scan_parquet("data/work/train_encoded_lf.parquet").filter(pl.col("uid") == 4121).filter(pl.col("item_id") == 876044).collect()  #28556

pl.scan_parquet("data/work/train_encoded_lf.parquet").filter(pl.col("uid") == 4121).filter(pl.col("event_type") == "listen").filter(pl.col("played_ratio_pct") > 90).collect()  #28556


# 4121 0.0 {789892, 754436, 119945, 124426, 689549, 374805, 603288, 587807, 307376, 165563, 870462, 575040, 918977, 268868, 455368, 662218, 833867, 452429, 128613, 185450, 879343, 748278} 
    
# [876044.0] [0.43005356192588806]

timestamp,is_organic,played_ratio_pct,track_length_seconds,event_type,uid,item_id
u32,u8,u16,u32,enum,i64,i64
137755,0,100,135,"""listen""",4121,390466
136685,0,100,155,"""listen""",4121,454787
136525,0,100,185,"""listen""",4121,805241
137260,0,100,215,"""listen""",4121,161244
135725,0,100,170,"""listen""",4121,61182
…,…,…,…,…,…,…
25467360,1,100,355,"""listen""",4121,273968
25463170,1,100,140,"""listen""",4121,846677
25465325,1,100,195,"""listen""",4121,212943


In [93]:
pl.scan_parquet("data/work/train_encoded_lf.parquet").filter(pl.col("item_id") == 218373).collect()  #28556
  # "1114058": 1114058,
  #   "2625039": 1114058,
  #   "4618546": 1114058,

timestamp,is_organic,played_ratio_pct,track_length_seconds,event_type,uid,item_id
u32,u8,u16,u32,enum,i64,i64
7843520,1,5,270,"""listen""",2143,218373


## Ранжирование

Ранжирование (Learning to Rank) — это задача, в которой модель не просто предсказывает значение, а упорядочивает объекты так, чтобы наиболее релевантные оказались выше в списке. Такой подход используется в поиске, рекомендациях и подборе контента — везде, где важна правильная сортировка результатов.

CatBoost хорошо подходит для ранжирования, потому что:

* поддерживает современные алгоритмы LTR (YetiRank, PairLogit и др.);
* отлично работает с категориальными признаками без дополнительной обработки;
* показывает высокую устойчивость и качество «из коробки»;
* легко использовать — достаточно передать группы (group_id).

Поэтому CatBoostRanker — удобная и мощная модель для построения систем рекомендаций и поиска.

### Гибридная модель

Объединием все полученные модели в одну гибридную. Каждая будет выдвать свой топ предсказаний ( но будем брать не 10 а больше). Затем полученные результаты будут уходить на вход модели ранжирования

топ 10 предсказания как правильно все разные - важно отобрать верно нужное

In [22]:
from catboost import CatBoostRanker
from utils.evaluate import filtering_listened_items


class HybridModel:
    def __init__(self):
        self.trend_model = MostPop_by_likes()
        self.new_items_model = NewItemsLastNDays()
        self.als = ALS()

        self.item_knn = ItemKNN()
        self.N = 10
        self.limit = 10
        self.test = False
        self.use_history = False

    
    def fit(self, train_df, train_df_conf, train_df_conf_als):
        print("Prepere history")



        
        history = (
            train_df
            .filter(pl.col("event_type") == "listen")
            # .filter(pl.col("is_organic") == 1)
            .select(["uid", "item_id", "timestamp"])
            .group_by(["uid", "item_id"])
            .agg(pl.col("timestamp").max().alias("last_timestamp"))
            .filter(pl.col("last_timestamp") >(60*60*24*300 - 60*60*24*7))
            .group_by("uid")
            .agg(pl.col("item_id"))      # без .list()
            .collect()
            
        )
        
        # превратить в dict {uid: [item_ids]}
        self.uid_to_items = dict(
            zip(
                history["uid"].to_list(),
                history["item_id"].to_list(),   # это уже списки item_id
            )
        )

        print("Prepere data")
        
        # материализуем, если LazyFrame
        df = train_df_conf.collect() if isinstance(train_df_conf, pl.LazyFrame) else train_df_conf
        
        # dict {(uid, item_id): conf}
        user_item_relevance = {
            (row["uid"], row["item_id"]): row["conf"]
            for row in df.select(["uid", "item_id", "conf"]).iter_rows(named=True)
        }
        
        # список уникальных uid
        hybrid_train_users = df["uid"].unique().to_list()
        
        print("Train recomended models")
        print("trend_model")
        self.trend_model.fit(train_df)
        self.new_items_model.fit(train_df) 

        train_df_conf_als = train_df_conf_als.with_columns(
                (pl.col("conf") != 0).cast(pl.Int8).alias("conf")
            )
        
        self.als.fit(train_df_conf_als)

        
        self.item_knn.fit(train_df)

        
        print("Prepere data for rancin model")
        
        X_rows = []      # сюда складываем фичи (строки)
        y_train = []     # сюда релевантность
        group_id = []    # группы (по пользователю)
        
        current_group = 0

        for user_id in tqdm(hybrid_train_users):
            # кандидаты от гибридной системы
            candidates = self.gather_candidates(user_id)
            
            if not candidates:
                # если модель не смогла ничего отдать – пропускаем пользователя
                continue
        
            rows = self.make_features(user_id, candidates)
            if not rows:
                continue

            n_rows = len(rows)
        
            for row in rows:
                item_id = row["item_id"]
        
                rel = user_item_relevance.get((user_id, item_id), 0.0)
                # print(rel)
                X_rows.append(row)
                y_train.append(rel)
        
            group_id.extend([current_group] * n_rows)
            current_group += 1
        
        self.X_train = pd.DataFrame(X_rows)
        self.y_train = y_train
        self.group_id = group_id
        
        print("Train rancin model")
        
        self.hybrid_model = CatBoostRanker(
            iterations=300,
            learning_rate=0.05,
            depth=6,
            loss_function="YetiRank",
            verbose=50,
        )
        
    def fit_ranker(self):
        self.hybrid_model.fit(
            self.X_train,
            self.y_train,
            group_id=self.group_id,
        )


    def gather_candidates(self, uid):

        candidates = {}
        
        rec, weights = self.trend_model.recommend(uid)
        rec, weights = rec[:self.limit], weights[:self.limit]
    
        for item, score in zip(rec, weights):
            candidates.setdefault(item, {})['trend'] = score
    
        rec, weights = self.new_items_model.recommend(uid)
        rec, weights = rec[:self.limit], weights[:self.limit]
    
        for item, score in zip(rec, weights):
            candidates.setdefault(item, {})['new'] = score

        rec, weights = self.als.recommend(uid)
        rec, weights = rec[:self.limit], weights[:self.limit]
    
        for item, score in zip(rec, weights):
            candidates.setdefault(item, {})['als'] = score

        rec, weights = self.item_knn.recommend(uid)
        rec, weights = rec[:self.limit], weights[:self.limit]
    
        for item, score in zip(rec, weights):
            candidates.setdefault(item, {})['item_knn'] = score
            
            
        return candidates

    def make_features(self, user_id, candidates):
        rows = []
        for item_id, scores in candidates.items():
    
            row = {
                "user_id": user_id,
                "item_id": item_id,
                "score_item_knn":   scores.get("item_knn", 0),
                "score_trend":      scores.get("trend", 0),
                "score_als":   scores.get("als", 0),
                "score_new":   scores.get("new", 0),
            }
    
            rows.append(row)
        
        return rows
            
    def recommend(self, uid):
        
        candidates = self.gather_candidates(uid)
        if not candidates:
           
            return []
        
        rows = self.make_features(uid, candidates)
        if not rows:
            return []
    
        df = pd.DataFrame(rows)
        if self.test:
            print("до", len(df))
        # user_listened_items = get_user_listened_items(self.history, uid)

        if self.use_history:
            if uid in self.uid_to_items:
                df = df[~df["item_id"].isin(self.uid_to_items[uid])]

        if self.test:
             print("после", len(df))
            
        scores = self.hybrid_model.predict(df)
        df["score"] = scores
    
        df_sorted = df.sort_values("score", ascending=False)
        
        return df_sorted["item_id"].tolist(), df_sorted["score"].tolist()  




In [12]:
import numpy as np
import pandas as pd

In [23]:

# ToDo формулу описать!!! зачем логорифирование и описать смысл коэффициентов почему max  
def calculate_conf_hyp(lf: pl.LazyFrame) -> pl.LazyFrame:
    lf = lf.filter(pl.col("played_ratio_max")>50)    
    return lf.with_columns(
        (
            20.0 * pl.col("like_flag").cast(pl.Float64)
            - 10.0 * pl.col("dislike_flag").cast(pl.Float64)
             +(
                pl.col("listen_count").cast(pl.Float64)
                * (pl.col("played_ratio_max").cast(pl.Float64) / 100.0)
            ).log1p()
        ).alias("conf")
    )

train_df_conf = pl.scan_parquet("data/work/train_df_conf.parquet")
train_df = pl.scan_parquet("data/work/train_df_preprocessed.parquet")
train_df_conf_hyp = merge_data_by_count(train_df)
train_df_conf_hyp = calculate_conf_hyp(train_df_conf_hyp)
# train_df_conf_hyp = train_df_conf_hyp.filter((pl.col("listen_count")>5) | (pl.col("like_flag")==1) )


# train_df_conf_hyp.collect()


hybrid = HybridModel()

hybrid.fit(train_df, train_df_conf_hyp, train_df_conf)
 


Prepere history
Prepere data
Train recomended models
trend_model


100%|███████████████████████████████████████████████████████████████████████| 15/15 [00:29<00:00,  1.98s/it, loss=0.00264]


Prepere data for rancin model


100%|█████████████████████████████████████████████████████████████████████████████████| 8896/8896 [10:46<00:00, 13.77it/s]


Train rancin model


In [78]:
hybrid.hybrid_model = CatBoostRanker(
            iterations=3000,
            learning_rate=0.1,
            l2_leaf_reg=0.3,
            depth=10,
            loss_function="YetiRank",
            # verbose=50,
            # logging_level='verbose_eval',
            verbose=True, 
            metric_period=50
    
        )
hybrid.fit_ranker()

0:	total: 148ms	remaining: 7m 24s
50:	total: 7.19s	remaining: 6m 55s
100:	total: 14.4s	remaining: 6m 54s
150:	total: 21.6s	remaining: 6m 48s
200:	total: 29s	remaining: 6m 44s
250:	total: 36.2s	remaining: 6m 36s
300:	total: 43.5s	remaining: 6m 29s
350:	total: 50.6s	remaining: 6m 22s
400:	total: 57.8s	remaining: 6m 14s
450:	total: 1m 4s	remaining: 6m 7s
500:	total: 1m 11s	remaining: 5m 58s
550:	total: 1m 18s	remaining: 5m 50s
600:	total: 1m 26s	remaining: 5m 44s
650:	total: 1m 33s	remaining: 5m 37s
700:	total: 1m 40s	remaining: 5m 30s
750:	total: 1m 47s	remaining: 5m 23s
800:	total: 1m 55s	remaining: 5m 15s
850:	total: 2m 2s	remaining: 5m 8s
900:	total: 2m 9s	remaining: 5m 1s
950:	total: 2m 16s	remaining: 4m 55s
1000:	total: 2m 24s	remaining: 4m 48s
1050:	total: 2m 31s	remaining: 4m 41s
1100:	total: 2m 39s	remaining: 4m 35s
1150:	total: 2m 46s	remaining: 4m 27s
1200:	total: 2m 53s	remaining: 4m 20s
1250:	total: 3m 1s	remaining: 4m 13s
1300:	total: 3m 8s	remaining: 4m 5s
1350:	total: 3m 1

In [80]:
evaluate_model(hybrid, users_history, test_df) #.filter(pl.col("uid") < 1000)

100%|█████████████████████████████████████████████████████████████████████████████████| 4827/4827 [06:58<00:00, 11.55it/s]

Mean Recall@10: 0.041739717449840695
Mean NDCG@10: 0.08076224853666847


### Анализ ошибок 

Точность системы составляла 2.7
Были выбраны пользоваетли для которых есть пересечения из всего списка рекомендаций есть попадания.
Было обнаружено что они находятся на >10 индексах, а между ними в основном треки которые были недавно показаны уже пользователю
Был добавлен фильтр на последние показанные песни 

In [ ]:
# [679713, 368953, 492083, 587807, 77971, 584672, 661967, 272391, 673338, 617140]
# [679713, 77971, 492083, 833867, 574324, 305144, 85378, 95745, 17549, 758397]
# [477143, 238780, 537198, 316031, 413405, 629088, 491538, 880709, 6289, 711413]
# [909289, 413867, 535463, 797204, 539301, 297910, 380508, 111967, 389120, 39158]


# [6289, 316031, 238780, 909289, 711413, 477143, 413405, 39158, 537198, 629088]


In [70]:
from tqdm import tqdm
from utils.metrics import ndcg_at_k, recall_at_k
import numpy as np
import polars as pl

def filtering_listened_items(rec, weights, listened_items: set, k=None):
    filtered_rec = []
    filtered_weights = []

    for item, weight in zip(rec, weights):
        if item in listened_items:
            continue
        filtered_rec.append(item)
        filtered_weights.append(weight)
        if k is not None and len(filtered_rec) >= k:
            break

    return filtered_rec, filtered_weights
    

def evaluate_model(model, user_history, test_lf: pl.LazyFrame, k: int = 10):

    overall_recall = []
    overall_ndcg = []
    test_lf = test_lf.collect()
    for i in tqdm(range(len(test_lf))): 
        
        uid = test_lf[i, "uid"] 
        user_true = set(test_lf[i, "items"] )

        if not user_true:
            continue
        try:
            rec, weights = model.recommend(uid)
        except:
            rec, weights = [], []

        # listened_items = user_history.get(uid, set())
        # rec, weights = filtering_listened_items(rec, weights, listened_items, k=k)

        if not rec:
            continue

        recall = recall_at_k(rec, user_true, k)
        ndcg = ndcg_at_k(rec, user_true, k)
        # if ndcg>0:
        #     print(uid)
        overall_recall.append(recall)
        overall_ndcg.append(ndcg)

    print("Mean Recall@{}:".format(k), np.mean(overall_recall))
    print("Mean NDCG@{}:".format(k), np.mean(overall_ndcg))

In [71]:
evaluate_model(model, users_history, test_df)

100%|████████████████████████████████████████████████████████████████████████████████| 4827/4827 [00:11<00:00, 402.64it/s]

Mean Recall@10: 0.019398229229073992
Mean NDCG@10: 0.04234377787508108


In [67]:
real = set(pl.scan_parquet("data/work/test_df_preprocessed_for_eval.parquet").filter(pl.col("uid") == 1078).select("items").collect().to_series().to_list()[0])
pred = set(model.recommend(1078)[0][:10])

pred & real

{303294}

In [69]:
pl.scan_parquet("data/work/test_df_preprocessed.parquet").filter(pl.col("uid") == 1078).filter(pl.col("item_id") == 303294).collect()

timestamp,is_organic,played_ratio_pct,track_length_seconds,event_type,uid,item_id
u32,u8,u16,u32,enum,i64,i64
25974815,0,100,205,"""listen""",1078,303294
